In [11]:
from pymongo import MongoClient
import urllib.request as urllib2
import requests
from bs4 import BeautifulSoup
import time
import random
import chardet
import pandas as pd
import tushare as ts
import re
import json
from selenium import webdriver
import numpy as np

import pprint


In [2]:
data = ts.get_stock_basics()
stock_list = data[data['timeToMarket'] != 0]['name']
stock_list.head()

code
000416     民生控股
600462     九有股份
002570      贝因美
002264    新 华 都
600113     浙江东日
Name: name, dtype: object

In [3]:
stock_list.index[0]

'000416'

In [4]:
stock_list[0]

'民生控股'

In [5]:
import datetime
today_date = datetime.datetime.today().strftime('%Y-%m-%d')
today_date

'2018-03-19'

In [6]:
len(stock_list)

3499

In [7]:
ip = '127.0.0.1'
port = 27017

client = MongoClient(ip, port)
db = client.quantaxis
coll = db.fin_data


In [27]:
stock_id = '300100'#'300624' # '300571'

In [29]:
num = coll.find({'code': stock_id}).count()
num

1

0

In [2]:


ip = '127.0.0.1'
port = 27017

#client = MongoClient(ip, port)
#db = client.quantaxis
#coll = db.fin_data

# scrape financial data

# get stock list
#data = ts.get_stock_basics()
#stock_list = data[data['timeToMarket'] != 0]['name']

stock_id = '300571'
stock_name = ''

url = url_template.format(stock_id)

try:
    # compose request
    # when javascript rendered page, cannot use urllib
    # use selenium
    #driver = webdriver.Chrome()
    driver = webdriver.PhantomJS()
    start_time = time.time()
    driver.get(url)
    print('fetch page for stock {} by time: {}'.format(stock_id, time.time() - start_time))
    #time.sleep(2)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, "lxml")

    '''
    with open('html.txt', 'w', encoding='utf-8') as f:
        print(soup.prettify(), file=f)
    '''

    data_tbody = soup.find("div", class_="data_tbody")
    top_thead = data_tbody.find('table', class_='top_thead')
    tbody = data_tbody.find('table', class_='tbody')

    # process top head to get finance date first
    items = top_thead.findAll('div', class_='td_w')
    dates = []
    for item in items:
        dates.append(item.text.strip())

    # process table body
    rows = tbody.findAll('tr')
    finance_tbl = []
    assert(len(rows) == 17)
    for i, row in enumerate(rows):
        if i == 0:
            items = row.findAll('div', class_='td_w')
        else:
            items = row.findAll('td')
        row_list = [None if item.text.strip() == "--" else item.text.strip() for item in items ]
        finance_tbl.append(row_list)

    # total # 17

    data_tbl = [dates] + finance_tbl
    data_tbl_stack = np.column_stack(data_tbl)

    df = pd.DataFrame(data_tbl_stack, columns=column).set_index('date', drop=False)
    #df['date'] = pd.to_datetime(df['date'])
    #df.set_index('date', inplace=True)

    if 'date' in df.columns:
        df.date = df.date.apply(str)

    doc_dic ={
        'code' : stock_id,
        'name': stock_name,
        'last_release_date' : df.ix[0, 'date'],
        'last_update_date': datetime.datetime.today().strftime('%Y-%m-%d'),
        'fin_data' : json.loads(df.to_json(orient='records')) # turns df into json, then into dictionary
    }

    coll.insert(doc_dic)
except Exception as e:
    print('some error occurs when processing stock {} : {} '.format(stock_id, str(e)))
    error_list.append(stock_id)
    continue

finished_list.append(stock_id)



C:\Users\ben_msi\Anaconda2\envs\python3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [3]:
column = ['date'] + column_list
data_tbl = [dates] + finance_tbl
data_tbl_stack = np.column_stack(data_tbl)



In [6]:
#df = pd.DataFrame(data_tbl_stack, columns=column)
#df['date'] = pd.to_datetime(df['date'])
#df.set_index('date', inplace=True)
df = pd.DataFrame(data_tbl_stack, columns=column).set_index('date', drop=False)

In [8]:
df

,date,jbmgsy,jlr,jlrtbzzl,kfjlr,kfjlrtbzzl,yyzsr,yyzsrtbzzl,mgjzc,jzcsrl,jzcsyl-tb,zcfzbl,mgzbgjj,mgwfplr,mgjyxjl,xsmll,chzzl,xsjll
date,,,,,,,,,,,,,,,,,,
2017-12-31,2017-12-31,3.91,4.49亿,575.04%,4.49亿,591.35%,14.65亿,97.16%,9.28,52.67%,40.35%,48.76%,3.61,4.68,-0.78,48.90%,1.27,30.68%
2017-09-30,2017-09-30,2.83,3.12亿,694.66%,3.10亿,None,9.53亿,68.71%,8.26,41.66%,31.44%,38.26%,3.61,3.65,-1.05,50.67%,0.97,32.71%
2017-06-30,2017-06-30,1.29,1.36亿,660.81%,1.36亿,682.08%,4.94亿,28.53%,6.90,22.44%,16.42%,36.74%,3.61,2.18,-0.89,45.77%,0.67,27.54%
2017-03-31,2017-03-31,0.55,4972.22万,961.42%,4964.78万,977.51%,2.31亿,119.25%,6.25,12.47%,6.63%,37.28%,3.61,1.46,-0.08,38.73%,0.49,21.49%
2016-12-31,2016-12-31,0.74,6657.50万,134.50%,6498.80万,157.66%,7.43亿,-20.24%,4.15,20.75%,17.81%,53.49%,1.49,1.40,1.65,23.20%,2.00,8.96%
2016-09-30,2016-09-30,0.44,3923.54万,None,None,None,5.65亿,None,None,None,None,None,None,None,0.64,20.36%,None,6.95%
2016-06-30,2016-06-30,0.20,1786.84万,None,1739.14万,None,3.84亿,None,3.46,6.03%,5.74%,60.51%,1.49,0.87,0.32,17.52%,1.19,4.65%
2016-03-31,2016-03-31,0.05,468.45万,None,460.77万,None,1.06亿,None,None,1.62%,None,None,None,None,None,22.15%,None,4.44%
2015-12-31,2015-12-31,0.32,2838.99万,-5.91%,2522.26万,-6.34%,9.32亿,21.86%,3.20,10.90%,9.87%,63.70%,1.49,0.67,1.03,15.52%,2.68,3.05%


In [10]:
doc_dic ={
    'code' : stock_id,
    'last_date' : df.ix[0, 'date'],
    'fin_data' : json.loads(df.to_json(orient='records')) # turns df into json, then into dictionary
}

In [11]:
print(doc_dic)

{'code': '300618', 'fin_data': [{'mgwfplr': '4.68', 'yyzsrtbzzl': '97.16%', 'kfjlr': '4.49亿', 'mgzbgjj': '3.61', 'yyzsr': '14.65亿', 'kfjlrtbzzl': '591.35%', 'jzcsyl-tb': '40.35%', 'mgjzc': '9.28', 'jlrtbzzl': '575.04%', 'xsmll': '48.90%', 'jbmgsy': '3.91', 'xsjll': '30.68%', 'jlr': '4.49亿', 'zcfzbl': '48.76%', 'date': '2017-12-31', 'mgjyxjl': '-0.78', 'chzzl': '1.27', 'jzcsrl': '52.67%'}, {'mgwfplr': '3.65', 'yyzsrtbzzl': '68.71%', 'kfjlr': '3.10亿', 'mgzbgjj': '3.61', 'yyzsr': '9.53亿', 'kfjlrtbzzl': None, 'jzcsyl-tb': '31.44%', 'mgjzc': '8.26', 'jlrtbzzl': '694.66%', 'xsmll': '50.67%', 'jbmgsy': '2.83', 'xsjll': '32.71%', 'jlr': '3.12亿', 'zcfzbl': '38.26%', 'date': '2017-09-30', 'mgjyxjl': '-1.05', 'chzzl': '0.97', 'jzcsrl': '41.66%'}, {'mgwfplr': '2.18', 'yyzsrtbzzl': '28.53%', 'kfjlr': '1.36亿', 'mgzbgjj': '3.61', 'yyzsr': '4.94亿', 'kfjlrtbzzl': '682.08%', 'jzcsyl-tb': '16.42%', 'mgjzc': '6.90', 'jlrtbzzl': '660.81%', 'xsmll': '45.77%', 'jbmgsy': '1.29', 'xsjll': '27.54%', 'jlr': '1.3